In [180]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import lightgbm as lgb
import pickle
from datetime import date, timedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [3]:
# 获取沪深300指数当前（）成分股
ts_api = ts.pro_api('f75ac7a71ba8bf7118966e7fb1a0224dda2345fa0f54d1f7b010b91b')
hs300 = ts_api.index_weight(index_code='000300.SH', start_date='20230201', end_date='20230228')
hs300

,index_code,con_code,trade_date,weight
0,000300.SH,603195.SH,20230201,0.0581
1,000300.SH,003816.SZ,20230201,0.1058
2,000300.SH,300782.SZ,20230201,0.2138
3,000300.SH,601615.SH,20230201,0.2189
4,000300.SH,601138.SH,20230201,0.1882
...,...,...,...,...
295,000300.SH,000157.SZ,20230201,0.1440
296,000300.SH,000069.SZ,20230201,0.1083
297,000300.SH,000063.SZ,20230201,0.4250
298,000300.SH,000002.SZ,20230201,0.6124


In [157]:
# 以000001.SZ为例搭建XGBoost预测模型
data = ts_api.daily(ts_code='000001.SZ')
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20230227,13.75,13.88,13.68,13.69,13.86,-0.17,-1.2266,621461.93,8.548540e+05
1,000001.SZ,20230224,14.02,14.03,13.80,13.86,14.05,-0.19,-1.3523,729989.04,1.012653e+06
2,000001.SZ,20230223,14.00,14.32,13.98,14.05,14.02,0.03,0.2140,824490.68,1.165540e+06
3,000001.SZ,20230222,14.00,14.12,13.94,14.02,14.10,-0.08,-0.5674,638922.39,8.957425e+05
4,000001.SZ,20230221,14.06,14.20,13.92,14.10,14.15,-0.05,-0.3534,990494.99,1.393153e+06
...,...,...,...,...,...,...,...,...,...,...,...
5995,000001.SZ,19970926,20.78,20.89,20.10,20.82,20.74,0.08,0.3900,59529.00,1.222137e+05
5996,000001.SZ,19970925,21.02,21.26,20.45,20.74,20.45,0.29,1.4200,103770.00,2.155430e+05
5997,000001.SZ,19970924,19.80,20.60,19.60,20.45,19.19,1.26,6.5700,154245.00,3.113654e+05
5998,000001.SZ,19970923,20.00,20.20,18.80,19.19,20.22,-1.03,-5.0900,118363.00,2.307110e+05


In [158]:
for code in tqdm(hs300['con_code']):
    data = ts_api.daily(ts_code=code)
    open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
    high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
    low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
    close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
    vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
    amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
    X = []
    y = []
    for i in range(len(open_pct)-5):
        X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
        y.append(close_pct[i+5])
    model = lgb.LGBMRegressor(learning_rate=0.01, metric='l2')
    model.fit(X,y,eval_metric='l2')
    filename = 'model/{}.pickle'.format(code)
    pickle.dump(model, open(filename,'wb'))

5.834074937454268


2

In [186]:
codes = ','.join(hs300['con_code'].values)
codes
data = ts_api.daily(ts_code=codes, start_date=str(date.today()-timedelta(8)).replace('-',''), end_date=str(date.today()).replace('-',''))
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20230227,13.75,13.88,13.68,13.69,13.86,-0.17,-1.2266,621461.93,854854.041
1,000002.SZ,20230227,17.23,17.40,17.16,17.18,17.22,-0.04,-0.2323,369424.10,637672.052
2,000063.SZ,20230227,29.88,31.67,29.84,31.29,30.10,1.19,3.9535,1139455.45,3536745.856
3,000069.SZ,20230227,5.30,5.32,5.22,5.25,5.30,-0.05,-0.9434,283076.97,149044.582
4,000100.SZ,20230227,4.41,4.48,4.40,4.42,4.45,-0.03,-0.6742,1338367.64,594035.499
...,...,...,...,...,...,...,...,...,...,...,...
1795,688363.SH,20230220,127.64,128.50,123.28,128.22,127.76,0.46,0.3601,37728.81,475480.441
1796,688396.SH,20230220,54.54,55.08,53.92,54.82,54.43,0.39,0.7165,29002.88,157970.181
1797,688561.SH,20230220,73.46,76.94,72.77,76.42,73.46,2.96,4.0294,26470.32,199705.245
1798,688599.SH,20230220,62.22,63.34,61.38,62.42,62.40,0.02,0.0321,154853.94,962066.706


In [207]:
p = {}
for code in tqdm(hs300['con_code']):
    day_data = data[data['ts_code']==code]
    open_pct = (100*day_data['open'].loc[::-1].diff().values[1:]/day_data['open'].loc[::-1].values[:-1]).tolist()
    high_pct = (100*day_data['high'].loc[::-1].diff().values[1:]/day_data['high'].loc[::-1].values[:-1]).tolist()
    low_pct = (100*day_data['low'].loc[::-1].diff().values[1:]/day_data['low'].loc[::-1].values[:-1]).tolist()
    close_pct = (100*day_data['close'].loc[::-1].diff().values[1:]/day_data['close'].loc[::-1].values[:-1]).tolist()
    vol_pct = (100*day_data['vol'].loc[::-1].diff().values[1:]/day_data['vol'].loc[::-1].values[:-1]).tolist()
    amount_pct = (100*day_data['amount'].loc[::-1].diff().values[1:]/day_data['amount'].loc[::-1].values[:-1]).tolist()
    X = [open_pct + high_pct + low_pct + close_pct + vol_pct + amount_pct]
    filename = 'model/{}.pickle'.format(code)
    model = pickle.load(open(filename,'rb'))
    p[model.predict(X)[0]] = code

100%|██████████| 300/300 [00:00<00:00, 421.05it/s]


In [213]:
[p[key] for key in sorted(p, reverse=True)[:10]]

['601689.SH',
 '688111.SH',
 '300661.SZ',
 '603882.SH',
 '605499.SH',
 '603369.SH',
 '002841.SZ',
 '002475.SZ',
 '601100.SH',
 '605117.SH']

In [215]:
[p[key] for key in sorted(p)[:10]]

['601698.SH',
 '601360.SH',
 '688363.SH',
 '688169.SH',
 '688005.SH',
 '002916.SZ',
 '688303.SH',
 '300316.SZ',
 '001979.SZ',
 '688561.SH']